In [1]:
import pandas as pd
import numpy as np
import os.path, sys
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegressionCV, LassoCV, LogisticRegression
import importlib
import pickle
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.spatial import ConvexHull
import seaborn as sns
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
name_mapping=pd.read_csv('/'.join(os.getcwd().split('/')[0:5])+ '/data/Map_1.csv', sep=',')
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis,)
from sklearn.svm import SVC
from EDdataset_GGZ import colsTypeCast
%load_ext autoreload
from GetDataReady import getDataNormalized

/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/
/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling


In [2]:
choice_dict={1.0: 'Core-DT, 5Cls', 1.1: 'Core-DT, only ED', 1.2: 'Core-DT, 3 ED and Others',
    2.0: 'Core-DT-EDEQ subscale, 5Cls', 2.1: 'Core-DT-EDEQ subscale, only ED', 2.2: 'Core-DT-EDEQ subscale, 3 ED and Others',
    3.0: 'Core, 5Cls', 3.1: 'Core, only ED', 3.2: 'Core, 3 ED and Others'}
save_dict={1.0: 'Core-DT-5Cls', 1.1: 'Core-DT-onlyED', 1.2: 'Core-DT-3ED-Others',
    2.0: 'Core-DT-EDEQ-5Cls', 2.1: 'Core-DT-EDEQ-only-ED', 2.2: 'Core-DT-EDEQ-3ED-Others',
    3.0: 'Core-5Cls', 3.1: 'Core-onlyED', 3.2: 'Core-3ED-Others'}
use_permutation_imp=['KNN','LDA','QDA','LSVC','RSVC']
savepicklpath='%s/pickles/'%(os.getcwd())
dataset_types={}
for key, val in choice_dict.items():
    dataset_types[key]=getDataNormalized(key)
ftr_imp_pkls='%s/rearr_ftr_weights.pkl'%savepicklpath
if os.path.exists('%s/rearr_ftr_weights.pkl'%savepicklpath):
    with open('%s/rearr_ftr_weights.pkl'%savepicklpath, 'rb') as f: 
        fimp_all_exps=pickle.load(f)
if os.path.exists('%s/rearr_prots_lvq.pkl'%savepicklpath):
    with open('%s/rearr_prots_lvq.pkl'%savepicklpath, 'rb') as f:  # open a text file
        prots_lvq_all=pickle.load(f) 

1.0: For dataset with Core, DT, and 5 classes (Ndims=24):

1.1: For dataset with Core, DT and only ED classes:

1.2: For dataset with Core, DT and 3 ED classes and Others:

2.0: For dataset with Core, DT, EDEQ subscales and all 5 classes (Ndim=27):

2.1: For dataset with Core, DT, EDEQ subscales, and only ED classes:

2.2: For dataset with Core, DT, EDEQ subscales and 3 ED classes and Others:

3.0: For dataset with Core only, and all 5 classes (Ndim=16):

3.1: For dataset with Core only, and only ED classes:

3.2: For dataset with Core, and 3 ED classes and Others:



In [3]:
choice=1.1
model_fname='%s_clf2.pkl'%dataset_type
modelsClassify='%s%s'%(savepicklpath, model_fname)
pipeClassifiers_all = pickle.load(open(modelsClassify, "rb"))
select_keys1, classifier_name1=['RF', 'KNN','RSVC', 'LDA','GMLVQ'],['Random Forest', 'K-nearest neighbour', 'SVM w/ RBF',
                                                                    'Linear Discr. Analysis','Generalized Matrix LVQ']
select_keys2, exp_name=['GNB','LGMLVQ2', 'LogLASSO'], save_dict[choice]
X, Y=dataset_types[choice]['zXtrain'], dataset_types[choice]['Ytrain']
Xtest=dataset_types[choice]['zXtest']
X.rename(columns=dict(zip(name_mapping['Current'].to_list(), name_mapping['Updated'].to_list())), inplace=True)
Xtest.rename(columns=dict(zip(name_mapping['Current'].to_list(), name_mapping['Updated'].to_list())), inplace=True)
nclasses, fimp_all=len(np.unique(Y)), fimp_all_exps[choice]
adapted_combo_cols=X.columns
ind = np.arange(len(adapted_combo_cols)-1)
mean_name, std_name, show_col_names=[],[], []
for idx, col in enumerate(adapted_combo_cols):
    if (col.split('-')[0]=='Main') | (col=='DT-BMI'):
        col=col.split('-')[1]   
    show_col_names.append(' '.join(col.split('_')))
    col='('+col+')'
    all_parts=' '.join(col.split('_'))    #temp='mean '+ all_parts
    mean_name.append('mean '+ all_parts)# temp='std '+ all_parts
    std_name.append('std '+ all_parts)
mpl.rcParams['hatch.linewidth'] = 0.5
res_df_colname=np.column_stack((mean_name, std_name)).flatten()
patterns = [ "/" , "++", "..", "xx", "\\" , "+" , "o", "O", "*", "|"  ]
colors=[np.array([228,26,28])/256, np.array([200,0,210])/256, np.array([77,175,74])/256, np.array([56,108,176])/256, 
        np.array([255,127,0]) /256, np.array([191,91,23])/256, np.array([231,41,138])/256, np.array([247,129,191])/256]
fs, alpha, ind2=10,0.6, ind
print(exp_name)
print(np.unique(Y))

Core-DT-onlyED
['Anorexia nervosa' 'Binge-ED' 'Bulimia nervosa' 'Other ED']


In [ ]:
fig01, ax01=plt.subplots(2,4, figsize=(12,6), layout='constrained')
for keyClf, clf in pipeClassifiers_all.items():          
    if ~keyClf in ['LGMLVQ1', 'LGMLVQ2']:
        train_pred_labs, test_pred_labs=clf[iter].predict(zXTrain), clf[iter].predict(zXTest)
        transformedTrain, transformedTest=clf.transform(zXTrain), clf.transform(zXTest)
        cls_idx=np.median(transformedTrain[YTrain== k], axis=0), np.where(class_members)[0]
        if keyClf=='GMLVQ':
            prot=clf.prototypes_            
        else:
            prot=np.median(transformedTrain[YTrain], axis=1)
        pwdists=[np.linalg.norm(prot-row) for row in data_to_plot[cls_idx,:]]
        sort_idx=np.argsort(np.abs(pwdists))